# Data Analysis

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('/Users/seandiehl/Desktop/PBP_1819updated1.csv')
df.head()

,X1,GAME_ID,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,TIME,ASSIST_PLAYER_ID,ASSIST_COUNT,BLOCK_PLAYER_ID,BLOCK_COUNT,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_TOTAL,FREE_THROW_MADE,SHOT_PLAYER_ID,POINTS_SCORED,TOTAL_POINTS_SCORED,SHOT_MADE,SHOT_TYPE,STEAL_PLAYER_ID,STEAL_COUNT,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_PLAY_TIME,HOME_PLAYER_ID_2_PLAY_TIME,HOME_PLAYER_ID_3_PLAY_TIME,HOME_PLAYER_ID_4_PLAY_TIME,HOME_PLAYER_ID_5_PLAY_TIME,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,HOME_PLAYER_ID_1_EXP_STLS,HOME_PLAYER_ID_2_EXP_STLS,HOME_PLAYER_ID_3_EXP_STLS,HOME_PLAYER_ID_4_EXP_STLS,HOME_PLAYER_ID_5_EXP_STLS,AWAY_PLAYER_ID_1_EXP_STLS,AWAY_PLAYER_ID_2_EXP_STLS,AWAY_PLAYER_ID_3_EXP_STLS,AWAY_PLAYER_ID_4_EXP_STLS,AWAY_PLAYER_ID_5_EXP_STLS,HOME_PLAYER_ID_1_EXP_BLKS,HOME_PLAYER_ID_2_EXP_BLKS,HOME_PLAYER_ID_3_EXP_BLKS,HOME_PLAYER_ID_4_EXP_BLKS,HOME_PLAYER_ID_5_EXP_BLKS,AWAY_PLAYER_ID_1_EXP_BLKS,AWAY_PLAYER_ID_2_EXP_BLKS,AWAY_PLAYER_ID_3_EXP_BLKS,AWAY_PLAYER_ID_4_EXP_BLKS,AWAY_PLAYER_ID_5_EXP_BLKS
0,0,21801052,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!
1,1,21801052,201143,Al Horford,203999,Nikola Jokic,200794,Paul Millsap,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,21801052,200794,Paul Millsap,0,NaN,0,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200794.0,NaN,NaN,False,Shot,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,0.229583333,0.111712963,0.132824074,0.145138889,0.148657407,0.164490741,0.155694444,0.181203704,0.159212963,0.150416667,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889,0.050138889,0.04837963,0.044861111,0.018472222,0.018472222,0.033425926,0.027268519,0.056296296,0.030787037,0.037824074,0.010555556,0.014953704,0.007037037,0.007037037,0.011435185,0.009675926,0.014953704,0.011435185,0.012314815,0.009675926,0.003518519,0.004398148,0.011435185,0.002638889,0.007916667,0.003518519,0.001759259,0.007916667,0.010555556,0.006157407
3,3,21801052,1628369,Jayson Tatum,0,NaN,0,NaN,21,NaN,NaN,NaN,NaN,162

The first stat we are going to breakdown is points, because it is the most troublesome. The 'TOTAL_POINTS_SCORED' column does not accurately update the points scored at the free throw line. It does take them into account when the next field goal is scored, but in the interim, the 'TOTAL_POINTS_SCORED' column does not reflect the update. Say, for example, a player entered the game and their first points scored were two free throws, followed by a layup. Their corresponding 'TOTAL_POINTS_SCORED' column would read a value of zero, until they scored their first field goal, at which point it would return four.

Another factor that needs to be taken into account is that substitutions can displace the player_ids. For example, a player_id can shift from 'HOME_PLAYER_ID_1' to 'HOME_PLAYER_ID_2' if a substitution was made. This is no good. It means we can't calculate the actual statistics across each of the player_id columns, because the values can shift. In order to combat this, we are going to create a column for each player_id, that way we can subsequently match the player_ids to the corresponding column. 

I am going to load in a smaller dataframe with only the values relevant for the points statistics, along with each of the player id's.

## Points (Explanation and Commentary for Calculating Expected vs. Actual Points)

In [3]:
points = pd.read_csv('/Users/seandiehl/Desktop/actual_points.csv')
points.head()


,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,21801052,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

The following funuction will be used to generate accurate total points scored for each of the player_ids. 

In [4]:

def apply_points(player_id):
    
    # note 1
    points.loc[(points['TIME']==0), str(player_id)] = 0
    # note 2
    points.loc[((points['SHOT_MADE']==True)&(points['SHOT_PLAYER_ID']==player_id)), str(player_id)] = points['TOTAL_POINTS_SCORED']
    # note 3
    points[str(player_id)] = points[str(player_id)].ffill()
    # note 4
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==1)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==2)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax() 
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==3)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    #note 5
    points[str(player_id)] = points[str(player_id)].shift(1)
    


Note 1 - this locates all the rows in the dataset where the time is equal to zero, and sets the player's score equal to zero as well, since a player can't have any points at the start of the game. <br>
Note 2 - this locates all the points where the shot is made and the shooting player matches the appropriate player id - then, it updates their the player's column appropriately <br>
Note 3 - this forward fills all the points until the next recorded value. So, starting at the zeros, the player's column will return zero until they make their first shot, when those points will be filled forward until their next shot. <br>
Note 4 - taking the forward filled points, we want to grab the first free throw of a series and add one, then we want to take the maximum value of points and fill that forward for each game. This is then repeated for free throw numbers 2 and 3. <br>
Note 5 - finally, we are going to shift the points column down by one value. because the total points column is updated on the same row the player takes the shot, we need to shift everything down by one row. Otherwise, when we do our final analysis by filtering makes and misses, the "Heat" variable will inaccurately reflect the difference between Actual and Expected points when the shots are taken. <br>

In [ ]:
for column in points.columns[27:]:
    player_id = int(column)
    apply_points(player_id)

In [ ]:
# points.to_csv('/Users/seandiehl/Desktop/actual_points_updated_ids.csv')

Now, I am going to hop over to excel to udpate the column values for the player_id's -- I'm sure there's a more efficient way to do it in python, but I know how to do it more quickly in excel, so that's what I'll stick with for this portion. 

In [ ]:
points_upd = pd.read_csv('/Users/seandiehl/Desktop/actual_points_updated_ids.csv')

Now that we've loaded it back in, lets add the expected points column to the updated dataframe and investigate.

In [ ]:
points_upd[['HOME_PLAYER_ID_1_EXP_PTS', 'HOME_PLAYER_ID_2_EXP_PTS', 'HOME_PLAYER_ID_3_EXP_PTS', 'HOME_PLAYER_ID_4_EXP_PTS', 'HOME_PLAYER_ID_5_EXP_PTS', 'AWAY_PLAYER_ID_1_EXP_PTS', 'AWAY_PLAYER_ID_2_EXP_PTS','AWAY_PLAYER_ID_3_EXP_PTS', 'AWAY_PLAYER_ID_4_EXP_PTS', 'AWAY_PLAYER_ID_5_EXP_PTS']] = df[['HOME_PLAYER_ID_1_EXP_PTS', 'HOME_PLAYER_ID_2_EXP_PTS', 'HOME_PLAYER_ID_3_EXP_PTS', 'HOME_PLAYER_ID_4_EXP_PTS', 'HOME_PLAYER_ID_5_EXP_PTS', 'AWAY_PLAYER_ID_1_EXP_PTS', 'AWAY_PLAYER_ID_2_EXP_PTS','AWAY_PLAYER_ID_3_EXP_PTS', 'AWAY_PLAYER_ID_4_EXP_PTS', 'AWAY_PLAYER_ID_5_EXP_PTS']]
points_upd.head()

,Unnamed: 0,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,HOME_PLAYER_ID_5_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS
0,0,21801052,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

The next step is to subtract the actual points from the expected points.

In [ ]:
points_upd = points_upd.astype(float)
# points_upd.replace(to_replace='#VALUE!', value=np.nan, inplace=True)
# points_upd.replace(to_replace="NaN", value=np.nan)
points_upd['HOME_PLAYER_ID_1_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_1_ACT_PTS'] - points_upd['HOME_PLAYER_ID_1_EXP_PTS']
points_upd['HOME_PLAYER_ID_2_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_2_ACT_PTS'] - points_upd['HOME_PLAYER_ID_2_EXP_PTS']
points_upd['HOME_PLAYER_ID_3_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_3_ACT_PTS'] - points_upd['HOME_PLAYER_ID_3_EXP_PTS']
points_upd['HOME_PLAYER_ID_4_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_4_ACT_PTS'] - points_upd['HOME_PLAYER_ID_4_EXP_PTS']
points_upd['HOME_PLAYER_ID_5_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_5_ACT_PTS'] - points_upd['HOME_PLAYER_ID_5_EXP_PTS']
points_upd['AWAY_PLAYER_ID_1_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_1_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_1_EXP_PTS']
points_upd['AWAY_PLAYER_ID_2_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_2_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_2_EXP_PTS']
points_upd['AWAY_PLAYER_ID_3_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_3_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_3_EXP_PTS']
points_upd['AWAY_PLAYER_ID_4_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_4_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_4_EXP_PTS']
points_upd['AWAY_PLAYER_ID_5_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_5_ACT_POINTS'] - points_upd['AWAY_PLAYER_ID_5_EXP_PTS']




points_upd.head()

,Unnamed: 0,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,HOME_PLAYER_ID_5_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS,HOME_PLAYER_ID_1_PTS_DIFF,HOME_PLAYER_ID_2_PTS_DIFF,HOME_PLAYER_ID_3_PTS_DIFF,HOME_PLAYER_ID_4_PTS_DIFF,HOME_PLAYER_ID_5_PTS_DIFF,AWAY_PLAYER_ID_1_PTS_DIFF,AWAY_PLAYER_ID_2_PTS_DIFF,AWAY_PLAYER_ID_3_PTS_DIFF,AWAY_PLAYER_ID_4_PTS_DIFF,AWAY_PLAYER_ID_5_PTS_DIFF
0,0.0,21801052.0,202681.0,203935.0,201143.0,202694.0,1628369.0,1627750.0,2039

## Rebounds (Process and Explanation for all non-points stats)
Next, we've got to to the same thing as above, but for the four stats left. This is a bit easier, as there are no marginal updates to be made depending on free throws made - each statistic recorded adds to the total by one, so by setting the starting scores for each game to 0, we can simply fill forward based on the recorded statistics. Here, I'll be doing it for rebounds.

In [ ]:
rebounds = pd.DataFrame()
rebounds[['HOME_PLAYER_ID_1_EXP_RBS',	'HOME_PLAYER_ID_2_EXP_RBS',	'HOME_PLAYER_ID_3_EXP_RBS',	'HOME_PLAYER_ID_4_EXP_RBS',	'HOME_PLAYER_ID_5_EXP_RBS',	'AWAY_PLAYER_ID_1_EXP_RBS',	'AWAY_PLAYER_ID_2_EXP_RBS',	'AWAY_PLAYER_ID_3_EXP_RBS',	'AWAY_PLAYER_ID_4_EXP_RBS',	'AWAY_PLAYER_ID_5_EXP_RBS']] = df[['HOME_PLAYER_ID_1_EXP_RBS',	'HOME_PLAYER_ID_2_EXP_RBS',	'HOME_PLAYER_ID_3_EXP_RBS',	'HOME_PLAYER_ID_4_EXP_RBS',	'HOME_PLAYER_ID_5_EXP_RBS',	'AWAY_PLAYER_ID_1_EXP_RBS',	'AWAY_PLAYER_ID_2_EXP_RBS',	'AWAY_PLAYER_ID_3_EXP_RBS',	'AWAY_PLAYER_ID_4_EXP_RBS',	'AWAY_PLAYER_ID_5_EXP_RBS']]

In [ ]:
rebounds.head()

,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS
0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!
1,0,0,0,0,0,0,0,0,0,0
2,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889
3,0.038888889,0.044722222,0.077777778,0.059305556,0.060277778,0.042777778,0.031111111,0.116666667,0.07875,0.055416667
4,0.048148148,0.05537037,0.096296296,0.073425926,0.07462963,0.052962963,0.038518519,0.144444444,0.0975,0.068611111


In [ ]:
rebounds[['REBOUND_PLAYER_ID',	'REBOUND_TEAM',	'REBOUND_OFFENSIVE_COUNT',	'REBOUND_DEFENSIVE_COUNT']] = df[['REBOUND_PLAYER_ID',	'REBOUND_TEAM',	'REBOUND_OFFENSIVE_COUNT',	'REBOUND_DEFENSIVE_COUNT']]

In [ ]:
rebounds.head()

,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT
0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889,NaN,NaN,NaN,NaN
3,0.038888889,0.044722222,0.077777778,0.059305556,0.060277778,0.042777778,0.031111111,0.116666667,0.07875,0.055416667,1628369.0,NaN,0.0,1.0
4,0.048148148,0.05537037,0.096296296,0.073425926,0.07462963,0.052962963,0.038518519,0.144444444,0.0975,0.068611111,NaN,NaN,NaN,NaN


You'll notice that there is a column for "Rebound Team". We are going to assume that this is an issue with the video tracking system that has trouble identifying which player grabbed the ball on a rebound. The RMSE for the projected rebounds versus actual rebounds per 36 minutes came out to about 1.7 - a number that was certainly influenced by this innacuracy, but well within an acceptable range, as the overall RMSE percentage error, for each of the stats combined, falls at about 11%. 11% error won't help anyone win any money in Vegas, but with the intense variablity with which most players perform on a game-by-game basis (depending on factors like opponent strength, travel, individual matchups, expected variation, etc) across the entire season, the projections should provide a decently accurate representation of a player's expected statistics. 

Now, I'm going to hop over to excel to add in the player columns

In [ ]:
rebounds.to_csv('/Users/seandiehl/Desktop/rebounds.csv')

In [ ]:
rebounds_upd = pd.read_csv('/Users/seandiehl/Desktop/rebounds_upd.csv')

In [ ]:
rebounds_upd.head()

,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

Here, we are simply going to create a new column that sums the offensive and defensive rebounds to account for total rebounds, and add the game id and the time in the game.

In [ ]:
rebounds_upd['TOTAL_RBS'] = rebounds_upd['REBOUND_OFFENSIVE_COUNT'] + rebounds_upd['REBOUND_DEFENSIVE_COUNT']
rebounds_upd['GAME_ID'] = df['GAME_ID']
rebounds_upd['TIME'] = df['TIME']


Next, we are going to create a function that sets all total rebounds to zero when the game first starts, adds the total rebound count to the corresponding player id column, and then forward fills these values. <br>
If you are unfamiliar with a forward fill, this simply means that for each player id column, the blank values will be filled throughout the game with the most recent total rebound count.

In [ ]:

def apply_rebounds(player_id):
    
    
    rebounds_upd.loc[(rebounds_upd['TIME']==0), str(player_id)] = 0
    
    rebounds_upd.loc[(rebounds_upd['REBOUND_PLAYER_ID']==player_id), str(player_id)] = rebounds_upd['TOTAL_RBS']
    rebounds_upd[str(player_id)] = rebounds_upd[str(player_id)].ffill()


 




In [ ]:
for column in rebounds_upd.columns[15:-3]:
    player_id = int(column)
    apply_rebounds(player_id)

In [ ]:
rebounds_upd.to_csv('/Users/seandiehl/Desktop/rebounds_upd_players.csv')

In [ ]:
rebounds_upd[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']]

Now, I've got to add the player id columns for who is on the floor, a step I had forgotten earlier

In [ ]:
rebounds_upd.head()

,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TOTAL_RBS,GAME_ID,TIME,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
0,0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

This next line of code will replace a default error message raised in excel with a numpy compatible blank value, then save the entire dataframe as the same datatype to ensure no errors are raised.

In [ ]:
rebounds_upd = rebounds_upd.replace(to_replace="#VALUE!", value=np.nan)
rebounds_upd = rebounds_upd.astype(float)

Back over to excel to fill the home and away team rebounds.

In [ ]:
rebounds_upd.to_csv('/Users/seandiehl/Desktop/rebounds_upd_players_1.csv')

In [ ]:
rebounds_upd_1 = pd.read_csv('/Users/seandiehl/Desktop/rebounds_upd_players_1.csv')

In [ ]:
rebounds_upd_1.head()

,Unnamed: 0.1,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TOTAL_RBS,GAME_ID,TIME,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_RBS,HOME_PLAYER_ID_2_ACT_RBS,HOME_PLAYER_ID_3_ACT_RBS,HOME_PLAYER_ID_4_ACT_RBS,HOME_PLAYER_ID_5_ACT_RBS,AWAY_PLAYER_ID_1_ACT_RBS,AWAY_PLAYER_ID_2_ACT_RBS,AWAY_PLAYER_ID_3_ACT_RBS,AWAY_PLAYER_ID_4_ACT_RBS,AWAY_PLAYER_ID_5_ACT_RBS
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And now we are going to account for the difference between the actual rebound count and the expected rebound count. 

In [ ]:
rebounds_upd_1['HOME_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_1_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_2_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_3_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_4_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_5_EXP_RBS']

rebounds_upd_1['AWAY_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_1_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_2_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_3_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_4_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_5_EXP_RBS']




## Other Statistics
Next, we are going to fill the remaining 3 statistics, although the process is the same as above for rebounds, so I'll spare the commentary. Scroll down to the next heading for the following analysis.

In [ ]:
assists = pd.DataFrame()
assists[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]
assists[['ASSIST_PLAYER_ID', 'ASSIST_COUNT']] = df[['ASSIST_PLAYER_ID', 'ASSIST_COUNT']]

In [ ]:
assists[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]] = df[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]] = df[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]]

In [ ]:
assists.to_csv('/Users/seandiehl/Desktop/assists.csv')

In [ ]:
assists_players = pd.read_csv('/Users/seandiehl/Desktop/assists_players.csv')
assists_players['TIME'] = df['TIME']

In [ ]:
assists_players.head()

,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,ASSIST_PLAYER_ID,ASSIST_COUNT,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TIME
0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
def apply_assists(player_id):
   
    
    # note 1
    assists_players.loc[(assists_players['TIME']==0), str(player_id)] = 0

    assists_players.loc[(assists_players['ASSIST_PLAYER_ID']==player_id), str(player_id)] = assists_players['ASSIST_COUNT']

    assists_players[str(player_id)] = assists_players[str(player_id)].ffill()

In [ ]:
for column in assists_players.columns[23:-1]:
    player_id = int(column)
    apply_assists(player_id)

In [ ]:
assists_players.to_csv('/Users/seandiehl/Desktop/assists_players_upd.csv')

In [ ]:
assist_players_upd = pd.read_csv('/Users/seandiehl/Desktop/assists_players_upd.csv')

In [ ]:
assist_players_upd.head()

,Unnamed: 0.1,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,ASSIST_PLAYER_ID,ASSIST_COUNT,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,HOME_PLAYER_ID_1_ACT_AST,HOME_PLAYER_ID_2_ACT_AST,HOME_PLAYER_ID_3_ACT_AST,HOME_PLAYER_ID_4_ACT_AST,HOME_PLAYER_ID_5_ACT_AST,AWAY_PLAYER_ID_1_ACT_AST,AWAY_PLAYER_ID_2_ACT_AST,AWAY_PLAYER_ID_3_ACT_AST,AWAY_PLAYER_ID_4_ACT_AST,AWAY_PLAYER_ID_5_ACT_AST,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TIME
0,0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
assist_players_upd = assist_players_upd.replace(to_replace="#VALUE!", value=np.nan)
assist_players_upd = assist_players_upd.astype(float)

In [ ]:
assist_players_upd['HOME_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_1_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_1_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_2_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_2_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_3_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_3_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_4_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_4_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_5_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_5_EXP_AST']

assist_players_upd['AWAY_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_1_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_1_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_2_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_2_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_3_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_3_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_4_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_4_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_5_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_5_EXP_AST']

In [ ]:
steals = pd.DataFrame()
steals[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]
steals[['STEAL_PLAYER_ID', 'STEAL_COUNT']] = df[['STEAL_PLAYER_ID', 'STEAL_COUNT']]
steals[['HOME_PLAYER_ID_1_EXP_STLS', 'HOME_PLAYER_ID_2_EXP_STLS', 'HOME_PLAYER_ID_3_EXP_STLS', 'HOME_PLAYER_ID_4_EXP_STLS', 'HOME_PLAYER_ID_5_EXP_STLS', 'AWAY_PLAYER_ID_1_EXP_STLS', 'AWAY_PLAYER_ID_2_EXP_STLS', 'AWAY_PLAYER_ID_3_EXP_STLS', 'AWAY_PLAYER_ID_4_EXP_STLS', 'AWAY_PLAYER_ID_5_EXP_STLS']] = df[['HOME_PLAYER_ID_1_EXP_STLS', 'HOME_PLAYER_ID_2_EXP_STLS', 'HOME_PLAYER_ID_3_EXP_STLS', 'HOME_PLAYER_ID_4_EXP_STLS', 'HOME_PLAYER_ID_5_EXP_STLS', 'AWAY_PLAYER_ID_1_EXP_STLS', 'AWAY_PLAYER_ID_2_EXP_STLS', 'AWAY_PLAYER_ID_3_EXP_STLS', 'AWAY_PLAYER_ID_4_EXP_STLS', 'AWAY_PLAYER_ID_5_EXP_STLS']]
steals['TIME'] = df['TIME']

In [ ]:
steals.head()

,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,STEAL_PLAYER_ID,STEAL_COUNT,HOME_PLAYER_ID_1_EXP_STLS,HOME_PLAYER_ID_2_EXP_STLS,HOME_PLAYER_ID_3_EXP_STLS,HOME_PLAYER_ID_4_EXP_STLS,HOME_PLAYER_ID_5_EXP_STLS,AWAY_PLAYER_ID_1_EXP_STLS,AWAY_PLAYER_ID_2_EXP_STLS,AWAY_PLAYER_ID_3_EXP_STLS,AWAY_PLAYER_ID_4_EXP_STLS,AWAY_PLAYER_ID_5_EXP_STLS,TIME
0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0
1,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0
2,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,0.010555556,0.014953704,0.007037037,0.007037037,0.011435185,0.009675926,0.014953704,0.011435185,0.012314815,0.009675926,19
3,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,0.011666667,0.016527778,0.007777778,0.007777778,0.012638889,0.010694444,0.016527778,0.012638889,0.013611111,0.010694444,21
4,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,0.014444444,0.020462963,0.00962963,0.00962963,0.015648148,0.013240741,0.020462963,0.015648148,0.016851852,0.013240741,26


In [ ]:
steals.to_csv('/Users/seandiehl/Desktop/steals.csv')

In [ ]:
steals_players = pd.read_csv('/Users/seandiehl/Desktop/steals.csv')

In [ ]:
steals_players.head()

,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,STEAL_PLAYER_ID,STEAL_COUNT,HOME_PLAYER_ID_1_EXP_STLS,HOME_PLAYER_ID_2_EXP_STLS,HOME_PLAYER_ID_3_EXP_STLS,HOME_PLAYER_ID_4_EXP_STLS,HOME_PLAYER_ID_5_EXP_STLS,AWAY_PLAYER_ID_1_EXP_STLS,AWAY_PLAYER_ID_2_EXP_STLS,AWAY_PLAYER_ID_3_EXP_STLS,AWAY_PLAYER_ID_4_EXP_STLS,AWAY_PLAYER_ID_5_EXP_STLS,TIME,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
def apply_steals(player_id):

    steals_players.loc[(steals_players['TIME']==0), str(player_id)] = 0
    steals_players.loc[(steals_players['STEAL_PLAYER_ID']==player_id), str(player_id)] = steals_players['STEAL_COUNT']
    steals_players[str(player_id)] = steals_players[str(player_id)].ffill()




In [ ]:
for column in steals_players.columns[24:]:
    player_id = int(column)
    apply_steals(player_id)

In [ ]:
steals_players.to_csv('/Users/seandiehl/Desktop/steals_players_upd.csv')

In [ ]:
steals_players_upd = pd.read_csv('/Users/seandiehl/Desktop/steals_players_upd.csv')

In [ ]:
steals_players_upd.head()

,Unnamed: 0.1,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,STEAL_PLAYER_ID,STEAL_COUNT,HOME_PLAYER_ID_1_EXP_STLS,HOME_PLAYER_ID_2_EXP_STLS,HOME_PLAYER_ID_3_EXP_STLS,HOME_PLAYER_ID_4_EXP_STLS,HOME_PLAYER_ID_5_EXP_STLS,AWAY_PLAYER_ID_1_EXP_STLS,AWAY_PLAYER_ID_2_EXP_STLS,AWAY_PLAYER_ID_3_EXP_STLS,AWAY_PLAYER_ID_4_EXP_STLS,AWAY_PLAYER_ID_5_EXP_STLS,TIME,HOME_PLAYER_ID_1_ACT_STL,HOME_PLAYER_ID_2_ACT_STL,HOME_PLAYER_ID_3_ACT_STL,HOME_PLAYER_ID_4_ACT_STL,HOME_PLAYER_ID_5_ACT_STL,AWAY_PLAYER_ID_1_ACT_STL,AWAY_PLAYER_ID_2_ACT_STL,AWAY_PLAYER_ID_3_ACT_STL,AWAY_PLAYER_ID_4_ACT_STL,AWAY_PLAYER_ID_5_ACT_STL,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
steals_players_upd = steals_players_upd.replace(to_replace="#VALUE!", value=np.nan)
steals_players_upd = steals_players_upd.astype(float)

In [ ]:
blocks = pd.DataFrame()
blocks[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]
blocks[['HOME_PLAYER_ID_1_EXP_BLKS',	'HOME_PLAYER_ID_2_EXP_BLKS',	'HOME_PLAYER_ID_3_EXP_BLKS',	'HOME_PLAYER_ID_4_EXP_BLKS',	'HOME_PLAYER_ID_5_EXP_BLKS',	'AWAY_PLAYER_ID_1_EXP_BLKS',	'AWAY_PLAYER_ID_2_EXP_BLKS',	'AWAY_PLAYER_ID_3_EXP_BLKS',	'AWAY_PLAYER_ID_4_EXP_BLKS',	'AWAY_PLAYER_ID_5_EXP_BLKS']] = df[['HOME_PLAYER_ID_1_EXP_BLKS',	'HOME_PLAYER_ID_2_EXP_BLKS',	'HOME_PLAYER_ID_3_EXP_BLKS',	'HOME_PLAYER_ID_4_EXP_BLKS',	'HOME_PLAYER_ID_5_EXP_BLKS',	'AWAY_PLAYER_ID_1_EXP_BLKS',	'AWAY_PLAYER_ID_2_EXP_BLKS',	'AWAY_PLAYER_ID_3_EXP_BLKS',	'AWAY_PLAYER_ID_4_EXP_BLKS',	'AWAY_PLAYER_ID_5_EXP_BLKS']]
blocks['TIME'] = df['TIME']


In [ ]:
blocks.head()

,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_EXP_BLKS,HOME_PLAYER_ID_2_EXP_BLKS,HOME_PLAYER_ID_3_EXP_BLKS,HOME_PLAYER_ID_4_EXP_BLKS,HOME_PLAYER_ID_5_EXP_BLKS,AWAY_PLAYER_ID_1_EXP_BLKS,AWAY_PLAYER_ID_2_EXP_BLKS,AWAY_PLAYER_ID_3_EXP_BLKS,AWAY_PLAYER_ID_4_EXP_BLKS,AWAY_PLAYER_ID_5_EXP_BLKS,TIME
0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0
1,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0,0,0,0,0,0,0,0,0,0,0
2,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.003518519,0.004398148,0.011435185,0.002638889,0.007916667,0.003518519,0.001759259,0.007916667,0.010555556,0.006157407,19
3,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.003888889,0.004861111,0.012638889,0.002916667,0.00875,0.003888889,0.001944444,0.00875,0.011666667,0.006805556,21
4,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.004814815,0.006018519,0.015648148,0.003611111,0.010833333,0.004814815,0.002407407,0.010833333,0.014444444,0.008425926,26


In [ ]:
blocks.to_csv('/Users/seandiehl/Desktop/blocks.csv')

In [ ]:
blocks_players = pd.read_csv('/Users/seandiehl/Desktop/blocks.csv')

In [ ]:
blocks_players['BLOCK_PLAYER_ID'] = df['BLOCK_PLAYER_ID']
blocks_players['BLOCK_COUNT'] = df['BLOCK_COUNT']

In [ ]:
blocks_players.head()

,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_EXP_BLKS,HOME_PLAYER_ID_2_EXP_BLKS,HOME_PLAYER_ID_3_EXP_BLKS,HOME_PLAYER_ID_4_EXP_BLKS,HOME_PLAYER_ID_5_EXP_BLKS,AWAY_PLAYER_ID_1_EXP_BLKS,AWAY_PLAYER_ID_2_EXP_BLKS,AWAY_PLAYER_ID_3_EXP_BLKS,AWAY_PLAYER_ID_4_EXP_BLKS,AWAY_PLAYER_ID_5_EXP_BLKS,TIME,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,BLOCK_PLAYER_ID,BLOCK_COUNT
0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
def apply_blocks(player_id):

    blocks_players.loc[(blocks_players['TIME']==0), str(player_id)] = 0
    blocks_players.loc[(blocks_players['BLOCK_PLAYER_ID']==player_id), str(player_id)] = blocks_players['BLOCK_COUNT']
    blocks_players[str(player_id)] = blocks_players[str(player_id)].ffill()

In [ ]:
for column in blocks_players.columns[22:-2]:
    player_id = int(column)
    apply_blocks(player_id)

In [ ]:
blocks_players.to_csv('/Users/seandiehl/Desktop/blocks_players_upd.csv')

In [ ]:
blocks_players_upd = pd.read_csv('/Users/seandiehl/Desktop/blocks_players_upd.csv')

In [ ]:
blocks_players_upd = blocks_players_upd.replace(to_replace="#VALUE!", value=np.nan)

blocks_players_upd = blocks_players_upd.astype(float)

## Heat Check (difference between expected and average statistics)

Here, we are going to incorporate all off the expected and average statistics for each player into one dataframe for analysis.

In [6]:
heat_analysis = pd.DataFrame()

In [ ]:
heat_analysis['HOME_PLAYER_ID_1_BLK_DIFF'] = blocks_players_upd['HOME_PLAYER_ID_1_ACT_BLKS'] - blocks_players_upd['HOME_PLAYER_ID_1_EXP_BLKS']
heat_analysis['HOME_PLAYER_ID_2_BLK_DIFF'] = blocks_players_upd['HOME_PLAYER_ID_2_ACT_BLKS'] - blocks_players_upd['HOME_PLAYER_ID_2_EXP_BLKS']
heat_analysis['HOME_PLAYER_ID_3_BLK_DIFF'] = blocks_players_upd['HOME_PLAYER_ID_3_ACT_BLKS'] - blocks_players_upd['HOME_PLAYER_ID_3_EXP_BLKS']
heat_analysis['HOME_PLAYER_ID_4_BLK_DIFF'] = blocks_players_upd['HOME_PLAYER_ID_4_ACT_BLKS'] - blocks_players_upd['HOME_PLAYER_ID_4_EXP_BLKS']
heat_analysis['HOME_PLAYER_ID_5_BLK_DIFF'] = blocks_players_upd['HOME_PLAYER_ID_5_ACT_BLKS'] - blocks_players_upd['HOME_PLAYER_ID_5_EXP_BLKS']

heat_analysis['AWAY_PLAYER_ID_1_BLK_DIFF'] = blocks_players_upd['AWAY_PLAYER_ID_1_ACT_BLKS'] - blocks_players_upd['AWAY_PLAYER_ID_1_EXP_BLKS']
heat_analysis['AWAY_PLAYER_ID_2_BLK_DIFF'] = blocks_players_upd['AWAY_PLAYER_ID_2_ACT_BLKS'] - blocks_players_upd['AWAY_PLAYER_ID_2_EXP_BLKS']
heat_analysis['AWAY_PLAYER_ID_3_BLK_DIFF'] = blocks_players_upd['AWAY_PLAYER_ID_3_ACT_BLKS'] - blocks_players_upd['AWAY_PLAYER_ID_3_EXP_BLKS']
heat_analysis['AWAY_PLAYER_ID_4_BLK_DIFF'] = blocks_players_upd['AWAY_PLAYER_ID_4_ACT_BLKS'] - blocks_players_upd['AWAY_PLAYER_ID_4_EXP_BLKS']
heat_analysis['AWAY_PLAYER_ID_5_BLK_DIFF'] = blocks_players_upd['AWAY_PLAYER_ID_5_ACT_BLKS'] - blocks_players_upd['AWAY_PLAYER_ID_5_EXP_BLKS']

In [ ]:
heat_analysis['HOME_PLAYER_ID_1_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_1_ACT_PTS'] - points_upd['HOME_PLAYER_ID_1_EXP_PTS']
heat_analysis['HOME_PLAYER_ID_2_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_2_ACT_PTS'] - points_upd['HOME_PLAYER_ID_2_EXP_PTS']
heat_analysis['HOME_PLAYER_ID_3_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_3_ACT_PTS'] - points_upd['HOME_PLAYER_ID_3_EXP_PTS']
heat_analysis['HOME_PLAYER_ID_4_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_4_ACT_PTS'] - points_upd['HOME_PLAYER_ID_4_EXP_PTS']
heat_analysis['HOME_PLAYER_ID_5_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_5_ACT_PTS'] - points_upd['HOME_PLAYER_ID_5_EXP_PTS']
heat_analysis['AWAY_PLAYER_ID_1_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_1_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_1_EXP_PTS']
heat_analysis['AWAY_PLAYER_ID_2_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_2_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_2_EXP_PTS']
heat_analysis['AWAY_PLAYER_ID_3_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_3_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_3_EXP_PTS']
heat_analysis['AWAY_PLAYER_ID_4_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_4_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_4_EXP_PTS']
heat_analysis['AWAY_PLAYER_ID_5_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_5_ACT_POINTS'] - points_upd['AWAY_PLAYER_ID_5_EXP_PTS']

In [ ]:
heat_analysis['HOME_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_1_EXP_RBS']
heat_analysis['HOME_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_2_EXP_RBS']
heat_analysis['HOME_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_3_EXP_RBS']
heat_analysis['HOME_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_4_EXP_RBS']
heat_analysis['HOME_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_5_EXP_RBS']
heat_analysis['AWAY_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_1_EXP_RBS']
heat_analysis['AWAY_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_2_EXP_RBS']
heat_analysis['AWAY_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_3_EXP_RBS']
heat_analysis['AWAY_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_4_EXP_RBS']
heat_analysis['AWAY_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_5_EXP_RBS']

In [ ]:
heat_analysis['HOME_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_1_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_1_EXP_AST']
heat_analysis['HOME_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_2_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_2_EXP_AST']
heat_analysis['HOME_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_3_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_3_EXP_AST']
heat_analysis['HOME_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_4_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_4_EXP_AST']
heat_analysis['HOME_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_5_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_5_EXP_AST']

heat_analysis['AWAY_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_1_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_1_EXP_AST']
heat_analysis['AWAY_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_2_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_2_EXP_AST']
heat_analysis['AWAY_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_3_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_3_EXP_AST']
heat_analysis['AWAY_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_4_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_4_EXP_AST']
heat_analysis['AWAY_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_5_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_5_EXP_AST']

In [ ]:
heat_analysis['HOME_PLAYER_ID_1_STL_DIFF'] = steals_players_upd['HOME_PLAYER_ID_1_ACT_STL'] - steals_players_upd['HOME_PLAYER_ID_1_EXP_STLS']
heat_analysis['HOME_PLAYER_ID_2_STL_DIFF'] = steals_players_upd['HOME_PLAYER_ID_2_ACT_STL'] - steals_players_upd['HOME_PLAYER_ID_2_EXP_STLS']
heat_analysis['HOME_PLAYER_ID_3_STL_DIFF'] = steals_players_upd['HOME_PLAYER_ID_3_ACT_STL'] - steals_players_upd['HOME_PLAYER_ID_3_EXP_STLS']
heat_analysis['HOME_PLAYER_ID_4_STL_DIFF'] = steals_players_upd['HOME_PLAYER_ID_4_ACT_STL'] - steals_players_upd['HOME_PLAYER_ID_4_EXP_STLS']
heat_analysis['HOME_PLAYER_ID_5_STL_DIFF'] = steals_players_upd['HOME_PLAYER_ID_5_ACT_STL'] - steals_players_upd['HOME_PLAYER_ID_5_EXP_STLS']

heat_analysis['AWAY_PLAYER_ID_1_STL_DIFF'] = steals_players_upd['AWAY_PLAYER_ID_1_ACT_STL'] - steals_players_upd['AWAY_PLAYER_ID_1_EXP_STLS']
heat_analysis['AWAY_PLAYER_ID_2_STL_DIFF'] = steals_players_upd['AWAY_PLAYER_ID_2_ACT_STL'] - steals_players_upd['AWAY_PLAYER_ID_2_EXP_STLS']
heat_analysis['AWAY_PLAYER_ID_3_STL_DIFF'] = steals_players_upd['AWAY_PLAYER_ID_3_ACT_STL'] - steals_players_upd['AWAY_PLAYER_ID_3_EXP_STLS']
heat_analysis['AWAY_PLAYER_ID_4_STL_DIFF'] = steals_players_upd['AWAY_PLAYER_ID_4_ACT_STL'] - steals_players_upd['AWAY_PLAYER_ID_4_EXP_STLS']
heat_analysis['AWAY_PLAYER_ID_5_STL_DIFF'] = steals_players_upd['AWAY_PLAYER_ID_5_ACT_STL'] - steals_players_upd['AWAY_PLAYER_ID_5_EXP_STLS']

In [ ]:
heat_analysis.head()

,HOME_PLAYER_ID_1_PTS_DIFF,HOME_PLAYER_ID_2_PTS_DIFF,HOME_PLAYER_ID_3_PTS_DIFF,HOME_PLAYER_ID_4_PTS_DIFF,HOME_PLAYER_ID_5_PTS_DIFF,AWAY_PLAYER_ID_1_PTS_DIFF,AWAY_PLAYER_ID_2_PTS_DIFF,AWAY_PLAYER_ID_3_PTS_DIFF,AWAY_PLAYER_ID_4_PTS_DIFF,AWAY_PLAYER_ID_5_PTS_DIFF,HOME_PLAYER_ID_1_RB_DIFF,HOME_PLAYER_ID_2_RB_DIFF,HOME_PLAYER_ID_3_RB_DIFF,HOME_PLAYER_ID_4_RB_DIFF,HOME_PLAYER_ID_5_RB_DIFF,AWAY_PLAYER_ID_1_RB_DIFF,AWAY_PLAYER_ID_2_RB_DIFF,AWAY_PLAYER_ID_3_RB_DIFF,AWAY_PLAYER_ID_4_RB_DIFF,AWAY_PLAYER_ID_5_RB_DIFF,HOME_PLAYER_ID_1_AST_DIFF,HOME_PLAYER_ID_2_AST_DIFF,HOME_PLAYER_ID_3_AST_DIFF,HOME_PLAYER_ID_4_AST_DIFF,HOME_PLAYER_ID_5_AST_DIFF,AWAY_PLAYER_ID_1_AST_DIFF,AWAY_PLAYER_ID_2_AST_DIFF,AWAY_PLAYER_ID_3_AST_DIFF,AWAY_PLAYER_ID_4_AST_DIFF,AWAY_PLAYER_ID_5_AST_DIFF,HOME_PLAYER_ID_1_STL_DIFF,HOME_PLAYER_ID_2_STL_DIFF,HOME_PLAYER_ID_3_STL_DIFF,HOME_PLAYER_ID_4_STL_DIFF,HOME_PLAYER_ID_5_STL_DIFF,AWAY_PLAYER_ID_1_STL_DIFF,AWAY_PLAYER_ID_2_STL_DIFF,AWAY_PLAYER_ID_3_STL_DIFF,AWAY_PLAYER_ID_4_STL_DIFF,AWAY_PLAYER_ID_5_STL_DIFF,HOME_PLAYER_ID_1_BLK_DIFF,HOME_PLAYER_ID_2_BLK_DIFF,HOME_PLAYER_ID_3_BLK_DIFF,HOME_PLAYER_ID_4_BLK_DIFF,HOME_PLAYER_ID_5_BLK_DIFF,AWAY_PLAYER_ID_1_BLK_DIFF,AWAY_PLAYER_ID_2_BLK_DIFF,AWAY_PLAYER_ID_3_BLK_DIFF,AWAY_PLAYER_ID_4_BLK_DIFF,AWAY_PLAYER_ID_5_BLK_DIFF
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-0.229583,-0.111713,-0.132824,-0.145139,-0.148657,-0.164491,-0.155694,-0.181204,-0.159213,-0.150417,0.035185,0.040463,0.070370,0.053657,0.054537,0.038704,0.028148,0.105556,0.07125,0.050139,-0.050139,-0.048380,-0.044861,-0.018472,-0.018472,-0.033426,-0.027269,-0.056296,-0.030787,-0.037824,-0.010556,-0.014954,-0.007037,-0.007037,-0.011435,-0.009676,-0.014954,-0.011435,-0.012315,-0.009676,-0.003519,-0.004398,-0.011435,-0.002639,-0.007917,-0.003519,-0.001759,-0.007917,-0.010556,-0.006157
3,-0.253750,-0.123472,-0.146806,-0.160417,-0.164306,-0.181806,-0.172083,-0.200278,-0.175972,-0.166250,0.038889,0.044722,0.077778,0.059306,0.060278,0.042778,0.031111,0.116667,0.07875,0.055417,-0.055417,-0.053472,-0.049583,-0.020417,-0.020417,-0.036944,-0.030139,-0.062222,-0.034028,-0.041806,-0.011667,-0.016528,-0.007778,-0.007778,-0.012639,-0.010694,-0.016528,-0.012639,-0.013611,-0.010694,-0.003889,-0.004861,-0.012639,-0.002917,-0.008750,-0.003889,-0.001944,-0.008750,-0.011667,-0.006806
4,-0.314167,-0.152870,-0.181759,-0.198611,-0.203426,-0.225093,-0.213056,-0.247963,-0.217870,-0.205833,0.048148,0.055370,0.096296,0.073426,0.074630,0.052963,0.038519,0.144444,0.09750,0.068611,-0.068611,-0.066204,-0.061389,-0.025278,-0.025278,-0.045741,-0.037315,-0.077037,-0.042130,-0.051759,-0.014444,-0.020463,-0.009630,-0.009630,-0.015648,-0.013241,-0.020463,-0.015648,-0.016852,-0.013241,-0.004815,-0.006019,-0.015648,-0.003611,-0.010833,-0.004815,-0.002407,-0.010833,-0.014444,-0.008426


Now that we have that dataframe, lets compile a metric for each of the players on the court that encapsulates their actual minus their expected statistics.

In [ ]:
heat_scores = pd.DataFrame()

In [ ]:
heat_scores['HOME_PLAYER_ID_1_UnAdj_Heat'] = heat_analysis['HOME_PLAYER_ID_1_PTS_DIFF'] + heat_analysis['HOME_PLAYER_ID_1_RB_DIFF'] + heat_analysis['HOME_PLAYER_ID_1_AST_DIFF'] + heat_analysis['HOME_PLAYER_ID_1_STL_DIFF'] + heat_analysis['HOME_PLAYER_ID_1_BLK_DIFF']
heat_scores['HOME_PLAYER_ID_2_UnAdj_Heat'] = heat_analysis['HOME_PLAYER_ID_2_PTS_DIFF'] + heat_analysis['HOME_PLAYER_ID_2_RB_DIFF'] + heat_analysis['HOME_PLAYER_ID_2_AST_DIFF'] + heat_analysis['HOME_PLAYER_ID_2_STL_DIFF'] + heat_analysis['HOME_PLAYER_ID_2_BLK_DIFF']
heat_scores['HOME_PLAYER_ID_3_UnAdj_Heat'] = heat_analysis['HOME_PLAYER_ID_3_PTS_DIFF'] + heat_analysis['HOME_PLAYER_ID_3_RB_DIFF'] + heat_analysis['HOME_PLAYER_ID_3_AST_DIFF'] + heat_analysis['HOME_PLAYER_ID_3_STL_DIFF'] + heat_analysis['HOME_PLAYER_ID_3_BLK_DIFF']
heat_scores['HOME_PLAYER_ID_4_UnAdj_Heat'] = heat_analysis['HOME_PLAYER_ID_4_PTS_DIFF'] + heat_analysis['HOME_PLAYER_ID_4_RB_DIFF'] + heat_analysis['HOME_PLAYER_ID_4_AST_DIFF'] + heat_analysis['HOME_PLAYER_ID_4_STL_DIFF'] + heat_analysis['HOME_PLAYER_ID_4_BLK_DIFF']
heat_scores['HOME_PLAYER_ID_5_UnAdj_Heat'] = heat_analysis['HOME_PLAYER_ID_5_PTS_DIFF'] + heat_analysis['HOME_PLAYER_ID_5_RB_DIFF'] + heat_analysis['HOME_PLAYER_ID_5_AST_DIFF'] + heat_analysis['HOME_PLAYER_ID_5_STL_DIFF'] + heat_analysis['HOME_PLAYER_ID_5_BLK_DIFF']

heat_scores['AWAY_PLAYER_ID_1_UnAdj_Heat'] = heat_analysis['AWAY_PLAYER_ID_1_PTS_DIFF'] + heat_analysis['AWAY_PLAYER_ID_1_RB_DIFF'] + heat_analysis['AWAY_PLAYER_ID_1_AST_DIFF'] + heat_analysis['AWAY_PLAYER_ID_1_STL_DIFF'] + heat_analysis['AWAY_PLAYER_ID_1_BLK_DIFF']
heat_scores['AWAY_PLAYER_ID_2_UnAdj_Heat'] = heat_analysis['AWAY_PLAYER_ID_2_PTS_DIFF'] + heat_analysis['AWAY_PLAYER_ID_2_RB_DIFF'] + heat_analysis['AWAY_PLAYER_ID_2_AST_DIFF'] + heat_analysis['AWAY_PLAYER_ID_2_STL_DIFF'] + heat_analysis['AWAY_PLAYER_ID_2_BLK_DIFF']
heat_scores['AWAY_PLAYER_ID_3_UnAdj_Heat'] = heat_analysis['AWAY_PLAYER_ID_3_PTS_DIFF'] + heat_analysis['AWAY_PLAYER_ID_3_RB_DIFF'] + heat_analysis['AWAY_PLAYER_ID_3_AST_DIFF'] + heat_analysis['AWAY_PLAYER_ID_3_STL_DIFF'] + heat_analysis['AWAY_PLAYER_ID_3_BLK_DIFF']
heat_scores['AWAY_PLAYER_ID_4_UnAdj_Heat'] = heat_analysis['AWAY_PLAYER_ID_4_PTS_DIFF'] + heat_analysis['AWAY_PLAYER_ID_4_RB_DIFF'] + heat_analysis['AWAY_PLAYER_ID_4_AST_DIFF'] + heat_analysis['AWAY_PLAYER_ID_4_STL_DIFF'] + heat_analysis['AWAY_PLAYER_ID_4_BLK_DIFF']
heat_scores['AWAY_PLAYER_ID_5_UnAdj_Heat'] = heat_analysis['AWAY_PLAYER_ID_5_PTS_DIFF'] + heat_analysis['AWAY_PLAYER_ID_5_RB_DIFF'] + heat_analysis['AWAY_PLAYER_ID_5_AST_DIFF'] + heat_analysis['AWAY_PLAYER_ID_5_STL_DIFF'] + heat_analysis['AWAY_PLAYER_ID_5_BLK_DIFF']

Now, we are going to add the shot player id, the shot made, and the home and away player ids.

In [ ]:
heat_scores[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]

heat_scores[['SHOT_PLAYER_ID', 'SHOT_MADE']] = df[['SHOT_PLAYER_ID', 'SHOT_MADE']]

In [ ]:
heat_scores.head()

,HOME_PLAYER_ID_1_UnAdj_Heat,HOME_PLAYER_ID_3_UnAdj_Heat,HOME_PLAYER_ID_4_UnAdj_Heat,HOME_PLAYER_ID_5_UnAdj_Heat,AWAY_PLAYER_ID_1_UnAdj_Heat,AWAY_PLAYER_ID_2_UnAdj_Heat,AWAY_PLAYER_ID_3_UnAdj_Heat,AWAY_PLAYER_ID_4_UnAdj_Heat,AWAY_PLAYER_ID_5_UnAdj_Heat,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,SHOT_PLAYER_ID,SHOT_MADE,HOME_PLAYER_ID_1_HEAT_ADJ,HOME_PLAYER_ID_2_UnAdj_Heat
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,-0.609185,0.000000
2,-0.258611,-0.125787,-0.119630,-0.131944,-0.172407,-0.171528,-0.151296,-0.141620,-0.153935,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,200794.0,False,-0.636697,-0.138981
3,-0.285833,-0.139028,-0.132222,-0.145833,-0.190556,-0.189583,-0.167222,-0.156528,-0.170139,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,-0.639593,-0.153611
4,-0.353889,-0.172130,-0.163704,-0.180556,-0.235926,-0.234722,-0.207037,-0.193796,-0.210648,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,201143.0,False,-0.646833,-0.190185


Exporting again to excel to match the heat scores to the shooters id.

In [ ]:
heat_scores.to_csv('/Users/seandiehl/Desktop/heat_scores.csv')

Here, the LoL does not stand for Laugh Out Loud, but rather, list of lists. This will allow us to iterate over each fo the players' heat scores and check out some descriptive statistics.

In [ ]:
heat_scores_LoL = heat_scores[[ 'HOME_PLAYER_ID_1_UnAdj_Heat',	'HOME_PLAYER_ID_3_UnAdj_Heat',	'HOME_PLAYER_ID_4_UnAdj_Heat',	'HOME_PLAYER_ID_5_UnAdj_Heat',	'AWAY_PLAYER_ID_1_UnAdj_Heat',	'AWAY_PLAYER_ID_2_UnAdj_Heat',	'AWAY_PLAYER_ID_3_UnAdj_Heat',	'AWAY_PLAYER_ID_4_UnAdj_Heat',	'AWAY_PLAYER_ID_5_UnAdj_Heat']].values.tolist()

In [ ]:
heat_scores_list = []
for i in heat_scores_LoL:
    for val in i:
        heat_scores_list.append(val)
        

This next line removes all the null values.

In [ ]:
heat_scores_list = [x for x in heat_scores_list if str(x) != 'nan']


Now, lets check out the mean and standard deviation of the heat scores. <br> 
We would expect the mean to be slightly positive. <br>
"Why?" you might ask... because coaches exist. They actively determine who is playing well, who is best suited for the matchups against the opponent, etc. <br>
As they actively manage the game, the players who are underperforming will see less court time, and players overperforming can subsequently be expected to see more. <br>
We also are going to hope for a pretty tight standard deviation, but one that is greater than the mean is positive, to ensure that our projections are still accurately reflecting actual - expected statistics. <br> In other words, we are hoping that players who are only slightly underperforming their expected averages aren't yanked out of the game.

In [ ]:
mean_heat = np.mean(heat_scores_list)
heat_stdev = np.std(heat_scores_list)

print("the average heat score is: ", mean_heat)
print("the st dev of heat is: ", heat_stdev)

the average heat score is:  3.071069926813965
the st dev of heat is:  5.041250348454235


Next, we are going to adjust each of the player's Heat scores to reflect their Heat as a Z-score. <br>
A Z-score represents a variable relative to the mean and standard deviation of the dataset. <br>
This allows us to determine just how "hot" or "cold" a player is based on the league average - in other words, players who are outperforming their expected average statistics by exactly 3.07 units will have a heat score of zer0, while a player underperforming by about 2 units will have a Z-score of -1.

In [ ]:




heat_scores['HOME_PLAYER_ID_1_HEAT_ADJ'] = (heat_scores['HOME_PLAYER_ID_1_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['HOME_PLAYER_ID_2_HEAT_ADJ'] = (heat_scores['HOME_PLAYER_ID_2_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['HOME_PLAYER_ID_3_HEAT_ADJ'] = (heat_scores['HOME_PLAYER_ID_3_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['HOME_PLAYER_ID_4_HEAT_ADJ'] = (heat_scores['HOME_PLAYER_ID_4_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['HOME_PLAYER_ID_5_HEAT_ADJ'] = (heat_scores['HOME_PLAYER_ID_5_UnAdj_Heat'] - mean_heat ) / heat_stdev


heat_scores['AWAY_PLAYER_ID_1_HEAT_ADJ'] = (heat_scores['AWAY_PLAYER_ID_1_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['AWAY_PLAYER_ID_2_HEAT_ADJ'] = (heat_scores['AWAY_PLAYER_ID_2_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['AWAY_PLAYER_ID_3_HEAT_ADJ'] = (heat_scores['AWAY_PLAYER_ID_3_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['AWAY_PLAYER_ID_4_HEAT_ADJ'] = (heat_scores['AWAY_PLAYER_ID_4_UnAdj_Heat'] - mean_heat ) / heat_stdev
heat_scores['AWAY_PLAYER_ID_5_HEAT_ADJ'] = (heat_scores['AWAY_PLAYER_ID_5_UnAdj_Heat'] - mean_heat ) / heat_stdev








Next, we are going to do a quick "sense check" on each of the columns' adjusted heat scores. We should see each column has a mean close to zero.

In [ ]:
print(heat_scores['HOME_PLAYER_ID_1_HEAT_ADJ'].mean())
print(heat_scores['HOME_PLAYER_ID_2_HEAT_ADJ'].mean())
print(heat_scores['HOME_PLAYER_ID_3_HEAT_ADJ'].mean())
print(heat_scores['HOME_PLAYER_ID_4_HEAT_ADJ'].mean())
print(heat_scores['HOME_PLAYER_ID_5_HEAT_ADJ'].mean())


-0.07539611652162706
-0.061623884062917564
0.11819818222902825
0.06492022174139933
0.015735057023865705


Next, I'm going to hop over to excel one last time to match the heat scores to the shooting player id, to calculate a new column "SHOOTERS_HEAT"<br>
It's certainly tiresome and another extra step, but as I mentioned, I'm intimately familiar with the index match functionality in excel, and not as familiar with the loc match on columns in pandas, so its quicker for me.

In [ ]:
heat_scores.to_csv('/Users/seandiehl/Desktop/heat_scores_adj.csv')

# Analysis

In [4]:
analyzing_heat = pd.read_csv('/Users/seandiehl/Desktop/heat_scores_adj.csv')

In [9]:
analyzing_heat.head()

,Unnamed: 0,HOME_PLAYER_ID_1_UnAdj_Heat,HOME_PLAYER_ID_2_UnAdj_Heat,HOME_PLAYER_ID_3_UnAdj_Heat,HOME_PLAYER_ID_4_UnAdj_Heat,HOME_PLAYER_ID_5_UnAdj_Heat,AWAY_PLAYER_ID_1_UnAdj_Heat,AWAY_PLAYER_ID_2_UnAdj_Heat,AWAY_PLAYER_ID_3_UnAdj_Heat,AWAY_PLAYER_ID_4_UnAdj_Heat,AWAY_PLAYER_ID_5_UnAdj_Heat,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,SHOT_PLAYER_ID,SHOT_MADE,SHOOTERS_HEAT,HOME_PLAYER_ID_1_HEAT_ADJ,HOME_PLAYER_ID_2_HEAT_ADJ,HOME_PLAYER_ID_3_HEAT_ADJ,HOME_PLAYER_ID_4_HEAT_ADJ,HOME_PLAYER_ID_5_HEAT_ADJ,AWAY_PLAYER_ID_1_HEAT_ADJ,AWAY_PLAYER_ID_2_HEAT_ADJ,AWAY_PLAYER_ID_3_HEAT_ADJ,AWAY_PLAYER_ID_4_HEAT_ADJ,AWAY_PLAYER_ID_5_HEAT_ADJ
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188,-0.609188
2,2,-0.258611,-0.138981,-0.125787,-0.119630,-0.131944,-0.172407,-0.171528,-0.151296,-0.141620,-0.153935,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,200794.0,False,-0.637280,-0.660487,-0.636757,-0.634140,-0.632918,-0.635361,-0.643387,-0.643213,-0.639200,-0.637280,-0.639723
3,3,-0.285833,-0.153611,-0.139028,-0.132222,-0.145833,-0.190556,-0.189583,-0.167222,-0.156528,-0.170139,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,-0.665887,-0.639659,-0.636766,-0.635416,-0.638116,-0.646987,-0.646795,-0.642359,-0.640238,-0.642937
4,4,-0.353889,-0.190185,-0.172130,-0.163704,-0.180556,-0.235926,-0.234722,-0.207037,-0.193796,-0.210648,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,201143.0,False,-0.643332,-0.679387,-0.646914,-0.643332,-0.641661,-0.645004,-0.655987,-0.655748,-0.650257,-0.647630,-0.650973


First thing we want to examine is all the shots players took when their adjusted heat score is greater than 0. <br>
This will help us in the coming analysis, determining whether or not players can be expected to shoot better than league average when they are overperforming their expected statistics at a rate higher than league average.

In [78]:
outperforming_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] > 0) & (analyzing_heat['SHOT_MADE']==True)]

outperforming_makes = len(outperforming_makes)
outperforming_makes

35448

In [79]:
outperforming_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] > 0) & (analyzing_heat['SHOT_MADE']==False)]
outperforming_misses = len(outperforming_misses)
outperforming_misses

39790

In [82]:
outperforming_shots = outperforming_makes + outperforming_misses
outperforming_percentage = outperforming_makes / outperforming_shots
outperforming_percentage

0.471144900183418

Next, we'll do the same thing for underperfomance by grabbing all players with a heat value less than 0.

In [84]:
underperforming_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] < 0) & (analyzing_heat['SHOT_MADE']==True)]
underperforming_makes = len(underperforming_makes)

In [85]:
underperforming_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT']<0)&(analyzing_heat['SHOT_MADE']==False)]
underperforming_misses = len(underperforming_misses)

In [86]:
underperforming_shots = underperforming_misses + underperforming_makes
underperforming_percentage = underperforming_makes / underperforming_shots

Next, we can break these down into different "buckets," so to speak, to perform a more granular analysis. I'm expecting that on the high ends (3+ standard deviations), the hypothesis test concerning paired proportions might fall victim to the smaller sample sizes. 

In [94]:
ice_cold_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT']<= -3) & (analyzing_heat['SHOT_MADE']==False)]
ice_cold_misses = len(ice_cold_misses)

In [95]:
ice_cold_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT']<= -3) & (analyzing_heat['SHOT_MADE']==True)]
ice_cold_makes= len(ice_cold_makes)


In [96]:

ice_cold_total_shots = ice_cold_makes + ice_cold_misses
ice_cold_percentage = ice_cold_makes / ice_cold_total_shots
print(ice_cold_total_shots)
print(ice_cold_percentage)


49
0.40816326530612246


Wow, players underforming by a wide margin have only taken 49 shots throuhgout the season. My gut tells me this is most likely due to two main factors: 1) coaches take players out who are playing horribly, and 2) players who are playing horribly don't want to shoot.

We're gonna do the same thing for different "buckets" of standard deviations for both under and over-performance.

In [13]:
cold_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT']<= -2) & (analyzing_heat['SHOOTERS_HEAT']>=-3) & (analyzing_heat['SHOT_MADE']==False)]
print(len(cold_misses))

530


In [14]:
cold_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT']<= -2) & (analyzing_heat['SHOOTERS_HEAT']>=-3) & (analyzing_heat['SHOT_MADE']==True)]
print(len(cold_makes))

402


In [15]:
cold_percentage = len(cold_makes) / (len(cold_misses) + len(cold_makes))
cold_percentage

0.4313304721030043

In [98]:
hot_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 3) & (analyzing_heat['SHOT_MADE']==True)]
hot_makes = len(hot_makes)

In [99]:
hot_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 3) & (analyzing_heat['SHOT_MADE']==False)]
hot_misses = len(hot_misses)

In [100]:
hot_total_shots = hot_makes + hot_misses
hot_percentage = hot_makes / hot_total_shots

In [19]:
heating_up_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 2) & (analyzing_heat['SHOOTERS_HEAT'] <=3) & (analyzing_heat['SHOT_MADE']==False)]
print(len(heating_up_misses))

3453


In [20]:
heating_up_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 2) & (analyzing_heat['SHOOTERS_HEAT'] <= 3) & (analyzing_heat['SHOT_MADE']==True)]
print(len(heating_up_makes))

2952


In [21]:
heating_up_percentage = len(heating_up_makes) / (len(heating_up_makes) + len(heating_up_misses))
heating_up_percentage

0.46088992974238874

In [22]:
warm_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 1) & (analyzing_heat['SHOOTERS_HEAT'] <=2) & (analyzing_heat['SHOT_MADE']==True)]
print(len(warm_makes))

8901


In [23]:
warm_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] >= 1) & (analyzing_heat['SHOOTERS_HEAT'] <= 2) & (analyzing_heat['SHOT_MADE']==False)]
print(len(warm_misses))

10113


In [24]:
warm_percentage = len(warm_makes) / (len(warm_misses) + len(warm_makes))
warm_percentage

0.4681287472388766

In [25]:
cooling_makes = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] <= -1) &  (analyzing_heat['SHOOTERS_HEAT'] >= -2) & (analyzing_heat['SHOT_MADE']==True)]
print(len(cooling_makes))

7699


In [26]:
cooling_misses = analyzing_heat.loc[(analyzing_heat['SHOOTERS_HEAT'] <= -1) &  (analyzing_heat['SHOOTERS_HEAT'] >= -2) & (analyzing_heat['SHOT_MADE']==False)]
print(len(cooling_misses))

9567


In [27]:
cooling_percentage = len(cooling_makes) / (len(cooling_makes) + len(cooling_misses))
cooling_percentage

0.44590524730684583

In [75]:
total_makes = analyzing_heat.loc[(analyzing_heat['SHOT_MADE']==True)]
total_makes = len(total_makes)
total_makes

101062

In [76]:
total_misses = analyzing_heat.loc[(analyzing_heat['SHOT_MADE']==False)]
total_misses = len(total_misses)
total_misses

118396

In [74]:
total_shots = len(total_makes) + len(total_misses)
total_percentage = len(total_makes) / total_shots
total_percentage

0.460507249678754

## Determining Statistical Significance

Now we want to take this data and perform a hypothesis test concerning two systems of proportions to determine if the values are statistically significant. I.e., we want to check whether or not the differences in shooting percentages can be attributed to random chance, or if we can say with reasonable confidence that they are due to differences in the underlying datasets. We are going to compare each group to the totals for the season. Our p-value will be 0.05, meaning the Z-score we are looking for in the output will be +/- 1.96 to determine a 95% Confidence Interval. Our Null Hypothesis for each of these tests is that the two datasets are equal, i.e., there is no underlying difference in shooting percentage based on previous over- or underperformance on the court. 

First, we are going to create a function that outputs the z-score concerning two systems of proportions.

In [77]:
def z_score(shooting_percentage, shots_made, shots_taken):
    numerator = total_percentage - shooting_percentage
    phat = (shots_made + total_makes) / (shots_taken + total_shots)
    denominator = np.sqrt(
        (phat) * (1 - phat) * ( (1 / total_shots) + ( 1 / shots_taken )  )
    )
    z = numerator / denominator
    return z 

Outpeforming Statistical Significance Test

In [83]:
z_score(outperforming_percentage, outperforming_makes, outperforming_shots)

-5.049645756004367

Neat! We can reject the null hypothesis and say that players who are outperforming their expected averages can be expected to shoot better than league average.

Underperforming Statistical Significance Test

In [87]:
z_score(underperforming_percentage, underperforming_makes, underperforming_shots)

0.023461895345213767

Here, we can't reject the null hypothesis.  

"Ice Cold" Statistical Significance Test

In [97]:
z_score(ice_cold_percentage,ice_cold_makes, ice_cold_total_shots)

0.7350317575317833

Again, we will reject the null hypothesis - with only 49 total shots, the sample size is just too small to draw any meaningful conclusion.

"White Hot" Statistical Significance Test

In [101]:
z_score(hot_percentage, hot_makes, hot_total_shots)

-0.685180116835147

Again, for this sub-group, we are going to reject the null hypothesis. 

## Classification Test

Here, we are going to check whether or not the players' Heat values are a useful metric in classifying attempted field goals as a make or a miss. First, we will generate a classification algorithm that makes stratified predictions for makes/misses - because we have about a 46% total field goal percentage, it will predict a make about 46% of the time, and we should expect around a similar metric for accuracy (correct predictions / total predictions). Then, we are going to create a similar classification algorithm that takes our Heat variables into account. The hope here is that the heat variable - again, measuring a player's past over- underperformance relative to the league average - adds a statistically significant increase to the accuracy of the model. This would prove that holistic over- or underperformance on the court influences a field goal's expected outcome. To put it another way, player's previous experiences on the court can influence their field goal percentage. 

Here, we are not expecting the classification algorithm to determine makes and misses with an incredible level of accuracy - we only hope to show that if it improves the accuracy over a stratified prediction algorithm, it does so to a statistically significant degree. Using only heat, and not accounting for the myriad of other confounding variables, like shot distance, defender distance, score differential, travel, etc., can hardly be expected to provide the be-all-end-all of predicting a shot's outcome. We are only looking to see whether or not this would be a useful metric to incorporate further into more comprehensive models. 

First, we are going to create a new dataframe that only holds the shooter's heat score and whether or not the shot would go in.

In [9]:
classification_df = analyzing_heat[['SHOOTERS_HEAT', 'SHOT_MADE']]


In [10]:
classification_df.head()
print(len(classification_df))

582468


Here, we are going to drop all null values, so that we only have data for all of the shots taken for the season.

In [11]:
classification_df.dropna(inplace=True)

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_87237/275397707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df.dropna(inplace=True)


In [112]:
total_shots_class = len(classification_df)
total_shots_class

219458

Here, we are going to assign makes to a value of 1 and misses to a value of 2.

In [13]:
classification_df['MAKE'] = np.where(classification_df['SHOT_MADE']==True, 1, 0)

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_87237/481629901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df['MAKE'] = np.where(classification_df['SHOT_MADE']==True, 1, 0)


In [14]:
classification_df.head()


,SHOOTERS_HEAT,SHOT_MADE,MAKE
2,-0.637280,False,0
4,-0.643332,False,0
6,-0.695586,False,0
9,-0.727986,True,1
10,-0.699186,False,0


In [15]:
classification_df.drop(columns='SHOT_MADE', inplace=True)

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_87237/2080647354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df.drop(columns='SHOT_MADE', inplace=True)


In [16]:
classification_df.head()

,SHOOTERS_HEAT,MAKE
2,-0.637280,0
4,-0.643332,0
6,-0.695586,0
9,-0.727986,1
10,-0.699186,0


Here, we are going to import the DummyClassifier to make our baseline predictions. Again, we would expect its accuracy to be somewhere around 50%. 

In [17]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

Here, we are assigning the features variable to the shooter's heat score, and the target variable, what we want to predict, to the make column.

In [18]:
features, target = classification_df['SHOOTERS_HEAT'].to_list(), classification_df['MAKE'].to_list()

Now we are going to split our features and targets into training data and testing data. 

In [19]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=42)

Next, we are going to assigne the baseline model to create "stratified" predictions - proportional to the values of makes and misses.

In [20]:
dummy = DummyClassifier(strategy='stratified', random_state=0)

In [108]:
dummy.fit(features_train, target_train)

DummyClassifier(random_state=0, strategy='stratified')

Here, we'll get the the accuracy score for the model.

In [109]:
dummy_accuracy = dummy.score(features_test, target_test)
dummy_accuracy

0.5067893921443544

Just a touch better than 50%, just what we were expecting.

Now, we'll create and train a binary classification algorithm on the data. <br> <br>
Remember: we are not looking for a drastic improvement in accuracy. We are only looking for an improvement that demonstrates that Heat is a valuable metric.

In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import RandomizedSearchCV


In [27]:
classification_df.dropna(inplace=True)

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_87237/275397707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  classification_df.dropna(inplace=True)


Now we are going to assign the feature and target variables, and reshape the corresponding arrays so that the classification algorithm can make sense of the data.

In [28]:
features, target = classification_df['SHOOTERS_HEAT'].to_list(), classification_df['MAKE'].to_list()

In [29]:
features = np.asarray(features)


target = np.asarray(target)


In [43]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.15, random_state=42)

In [44]:
features_train = features_train.reshape(-1, 1)
target_train = target_train.reshape(-1, 1)
features_test = features_test.reshape(-1, 1)


features_train.shape

(186539, 1)

In [45]:
xgbc = XGBClassifier()

Because we are choosing a Boosted Random Forest Classification Algorithm with XGBoost Classifier, we can tune the algorithm the same way a mechanic would tune an engine to optimize performance. We are going to provide some parameters for the algorithm below, and run a cross-validated search on the data to provide the best parameters.

In [46]:
params = {
    "learning_rate" : [0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    "max_depth" : [3, 4, 5,6, 8, 10, 12, 15],
    "min_child_weight" : [1, 3, 5, 7],
    "gamma" : [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample_bytree" : [0.3, 0.4, 0.5, 0.7]
    
}

In [59]:
rs_model = RandomizedSearchCV(xgbc, param_distributions=params, n_iter=25, scoring='accuracy', n_jobs=-1, cv=10, verbose=0)

Now that we have our best parameters, we can assign them to our classification algorithm

In [61]:
xgbc = rs_model.best_estimator_
xgbc

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4,
              enable_categorical=False, gamma=0.2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

Now we are going to train the model

In [62]:
xgbc.fit(features_train, target_train)

[21:26:14] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4,
              enable_categorical=False, gamma=0.2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=5, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1,
              predictor='auto', random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None)

Cool! Now we can make predictions based on this data.

In [63]:
preds = xgbc.predict(features_test)

In [188]:
y_hat = xgbc.fit(features_train, target_train).predict(features_test)

/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sh

[19:35:53] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [110]:
heat_accuracy = xgbc.score(preds, target_test)
heat_accuracy

0.5403262553540509

And there it is! It looks like the Heat metric improved accuracy by about 4%. Now we can run a hypothesis test concerning two proportions and determine if the improvement in accuracy can be attributed to an underlying difference in the performance of the algorithms, or if the improvement can be attributed to random variation. Our null hypothesis here will be that the classification algorithms are the same, and we will again use a p-value of 0.05 to signify rejection of the null hypothesis. The alternative hypothesis in this case will be that the Heat metric does provide a useful insight for the classification of shots into makes and misses.

In [115]:
numerator = dummy_accuracy - heat_accuracy
phat = ((dummy_accuracy * total_shots_class) + (heat_accuracy * total_shots_class)) / (total_shots_class * 2)
denominator = np.sqrt( (phat) * (1 - phat) * ( (1/total_shots_class) + (1 / total_shots_class)) )
z = numerator / denominator
print(z)

-22.243120949232406


Wow! That z-score certainly allows us to reject our null hypothesis, and even allows us to postscript in a p-value of 0.01 for a 99% Confidence Interval. <br> <br>
This tells us that actual holistic performance compared to the expected holistic performance on the court, when normalized relative to the league, provides a useful metric for determining makes and misses.